In [1]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG

D:\zzyAnaconda\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
df=pd.read_csv("E:/git/dsmp-2024-group22/data_preprocess/DATA/processed_tapes.csv",index_col=0)
df=df.dropna()
df=df.sort_values(by='date')
df.rename(columns={'max_price': 'High', 'min_price': 'Low','open_price': 'Open','close_price': 'Close'}, inplace=True)
df

,date,High,Low,Open,Close
0,2025-01-02,303.0,243.0,267.0,288.0
1,2025-01-03,317.0,270.0,281.0,291.0
4,2025-01-06,346.0,276.0,278.0,327.0
5,2025-01-07,388.0,317.0,336.0,369.0
6,2025-01-08,374.0,303.0,365.0,311.0
...,...,...,...,...,...
174,2025-06-25,114.0,93.0,96.0,104.0
175,2025-06-26,114.0,94.0,113.0,111.0
176,2025-06-27,113.0,93.0,106.0,111.0
179,2025-06-30,113.0,93.0,103.0,110.0


In [3]:
df_predict=pd.read_csv("E:/git/dsmp-2024-group22/data_preprocess/DATA/LSTM_prediction.csv")
df_predict['Open'] = pd.to_numeric(df_predict['Open'], downcast='float')
df_predict['High'] = pd.to_numeric(df_predict['High'], downcast='float')
df_predict['Low'] = pd.to_numeric(df_predict['Low'], downcast='float')
df_predict

,Unnamed: 0,Open,High,Low,Close
0,2025-04-28-1,158.0,160.0,140.0,152.052260
1,2025-04-28-2,152.0,161.0,144.0,152.442180
2,2025-04-28-3,158.0,162.0,142.0,154.090730
3,2025-04-28-4,161.0,162.0,141.0,155.025620
4,2025-04-28-5,160.0,160.0,140.0,155.110200
...,...,...,...,...,...
2245,2025-07-01-46,109.0,113.0,93.0,110.052704
2246,2025-07-01-47,109.0,113.0,93.0,109.887670
2247,2025-07-01-48,109.0,113.0,93.0,109.707230
2248,2025-07-01-49,107.0,113.0,93.0,109.346500


In [4]:
filtered_df = df_predict[df_predict['Unnamed: 0'].str.endswith('-50')]
filtered_df = filtered_df.rename(columns={'Unnamed: 0': 'Date'})
filtered_df['Date'] = filtered_df['Date'].str.replace('-50', '', regex=True)
filtered_df['Date'] = pd.to_datetime(filtered_df['Date'])
filtered_df=filtered_df.reset_index(drop=True)
filtered_df

,Date,Open,High,Low,Close
0,2025-04-28,173.0,182.0,161.0,171.589100
1,2025-04-29,186.0,190.0,171.0,180.428390
2,2025-04-30,188.0,198.0,177.0,187.631350
3,2025-05-01,189.0,193.0,173.0,183.669860
4,2025-05-02,199.0,206.0,185.0,192.326070
5,2025-05-06,210.0,211.0,193.0,201.003200
6,2025-05-07,182.0,186.0,166.0,174.886490
7,2025-05-08,179.0,179.0,159.0,169.729810
8,2025-05-09,170.0,174.0,153.0,166.061170
9,2025-05-12,169.0,179.0,158.0,169.006520


In [5]:
df_v=pd.read_csv("E:/git/dsmp-2024-group22/data_preprocess/DATA/tapes_volume.csv")
df_v=df_v["Volume"][80:125]
df_v=df_v.reset_index(drop=True)
df_v=pd.DataFrame(df_v)
df_v

,Volume
0,56222
1,57055
2,56907
3,56511
4,56109
5,55601
6,56089
7,55379
8,55457
9,56891


In [6]:
filtered_df["Volume"]=df_v["Volume"]
filtered_df

,Date,Open,High,Low,Close,Volume
0,2025-04-28,173.0,182.0,161.0,171.589100,56222
1,2025-04-29,186.0,190.0,171.0,180.428390,57055
2,2025-04-30,188.0,198.0,177.0,187.631350,56907
3,2025-05-01,189.0,193.0,173.0,183.669860,56511
4,2025-05-02,199.0,206.0,185.0,192.326070,56109
5,2025-05-06,210.0,211.0,193.0,201.003200,55601
6,2025-05-07,182.0,186.0,166.0,174.886490,56089
7,2025-05-08,179.0,179.0,159.0,169.729810,55379
8,2025-05-09,170.0,174.0,153.0,166.061170,55457
9,2025-05-12,169.0,179.0,158.0,169.006520,56891


In [7]:
filtered_df=filtered_df.set_index("Date")
filtered_df

,Open,High,Low,Close,Volume
Date,,,,,
2025-04-28,173.0,182.0,161.0,171.589100,56222
2025-04-29,186.0,190.0,171.0,180.428390,57055
2025-04-30,188.0,198.0,177.0,187.631350,56907
2025-05-01,189.0,193.0,173.0,183.669860,56511
2025-05-02,199.0,206.0,185.0,192.326070,56109
2025-05-06,210.0,211.0,193.0,201.003200,55601
2025-05-07,182.0,186.0,166.0,174.886490,56089
2025-05-08,179.0,179.0,159.0,169.729810,55379
2025-05-09,170.0,174.0,153.0,166.061170,55457


In [12]:
class SmaCross(Strategy):
    n1 = 1
    n2 = 5

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()
bt = Backtest(filtered_df, SmaCross,
              cash=10000, commission=.002,
              exclusive_orders=True)

output = bt.run()
output

Start                     2025-04-28 00:00:00
End                       2025-07-01 00:00:00
Duration                     64 days 00:00:00
Exposure Time [%]                   84.444444
Equity Final [$]                     13566.21
Equity Peak [$]                   15569.65208
Return [%]                            35.6621
Buy & Hold Return [%]               -37.44624
Return (Ann.) [%]                  451.782343
Volatility (Ann.) [%]              451.014967
Sharpe Ratio                         1.001701
Sortino Ratio                       11.290013
Calmar Ratio                        27.168327
Max. Drawdown [%]                  -16.629008
Avg. Drawdown [%]                    -6.07522
Max. Drawdown Duration       28 days 00:00:00
Avg. Drawdown Duration       11 days 00:00:00
# Trades                                    9
Win Rate [%]                        44.444444
Best Trade [%]                      28.908096
Worst Trade [%]                     -5.406578
Avg. Trade [%]                    

In [13]:
bt.plot()

Row(id='2523', ...)